# Car Price Prediction - Web Scraping Part

In [5]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from bs4 import BeautifulSoup as bs
import requests
import re
import time
import numpy as np

In [6]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [19]:
url1 = 'https://www.cars.com/shopping/results/\
?page='
url2 = '&page_size=20&list_price_max=&makes[]=\
&maximum_distance=all&models[]=&stock_type=used&zip='

url3 = 'https://www.cars.com/shopping/results/\
?page='
url4 = '&page_size=20&list_price_max=&makes[]=&maximum_distance=all&models[]=&stock_type=new&zip='

url_list = []

for i in range(60):
    url_list.append(url1 + str(i+1) + url2)

column_names = ['name', 'price', 'mileage', 'ex_color', 'in_color', 'drivetrain',\
                'mpg', 'fuel_type', 'transmission', 'engine']    

df = pd.DataFrame(columns = column_names)

response_list = []

tmp_df = pd.DataFrame()

for url in url_list:
    ex_colors = []
    in_colors = []
    drivetrains = []
    mpgs = []
    fuel_types = []
    transmissions = []
    engines = []
    res = requests.get(url).text
    soup = bs(res)
    
    # getting names
    names = soup.find_all(name='h2', attrs={'class': 'title'})
    names = pd.DataFrame(names)
    tmp_df['name'] = names

    # getting prices
    prices = soup.find_all(name='span', attrs={'class': 'primary-price'})
    prices = pd.DataFrame(prices)
    prices = prices.replace('\D', '', regex=True)
    tmp_df['price'] = prices

    # getting mileages
    mileages = soup.find_all(name='div', attrs={'class': 'mileage'})
    mileages = pd.DataFrame(mileages)
    mileages = prices.replace('\D', '', regex=True)
    tmp_df['mileage'] = mileages

    # getting links
    link_list = soup.find_all(name='a', attrs={'class': 'vehicle-card-link js-gallery-click-link'})
    links = []
    for link in link_list:
        links.append('https://www.cars.com' + link.get('href'))
        

    for link in links:
        res = requests.get(link).text
        time.sleep(2)
        sp = bs(res)
        # adding exterior color
        ex_color = sp.find(name='dt', text='Exterior color')\
            .next.next.next.text
        ex_colors.append(ex_color)

        # adding interior color
        in_color = sp.find(name='dt', text='Interior color')\
            .next.next.next.text
        in_colors.append(in_color)

        # adding drivetrain
        dt = sp.find(name='dt', text='Drivetrain').next.next.next.text
        drivetrains.append(dt)

        # adding mpg
        spf = sp.find(name='a', attrs={'class': 'sds-tooltip__trigger'})
        if spf != None:
            mpg = spf.previous_sibling.previous_sibling.text
            mpgs.append(mpg)
        else:
            mpgs.append('None')

        # adding fuel type
        fuel_type = sp.find(name='dt', text='Fuel type').next.next.next.text
        fuel_types.append(fuel_type)

        # adding transmission
        transmission = sp.find(name='dt', text='Transmission')\
            .next.next.next.text
        transmissions.append(transmission)

        # adding engine
        engine = sp.find(name='dt', text='Engine').next.next.next.text
        engines.append(engine)
        
    tmp_df['ex_color'] = ex_colors
    tmp_df['in_color'] = in_colors
    tmp_df['drivetrain'] = drivetrains
    tmp_df['mpg'] = mpgs
    tmp_df['fuel_type'] = fuel_types
    tmp_df['transmission'] = transmissions
    tmp_df['engine'] = engines
    
    # df.append(tmp_df)
    
    df = pd.concat([df, tmp_df], axis=0)
    
    tmp_df.drop(tmp_df.index,inplace=True)

df = df.reset_index()

# response1 = requests.get(url1 + '1' + url2)

# soup = bs(response_list[0])

In [20]:
# df.drop(['index'])
df

,index,name,price,mileage,ex_color,in_color,drivetrain,mpg,fuel_type,transmission,engine
0,0,2019 Honda Accord Sport,27995,27995,White,Red,Front-wheel Drive,29–35,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo
1,1,2018 Porsche Macan,41998,41998,Gray,Black,All-wheel Drive,20–25,Gasoline,7-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo
2,2,2015 Chevrolet Corvette Stingray Z51,52000,52000,Velocity Yellow Tinted Clearcoat,Black,Rear-wheel Drive,17–29,Gasoline,8-Speed Automatic,6.2L V8 16V GDI OHV
3,3,2020 Jeep Grand Cherokee Overland,41998,41998,Ivory,Sepia / Black,Four-wheel Drive,18–25,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC
4,4,2017 Ford Mustang GT Premium,29995,29995,Ingot Silver Metallic,Ebony w/Metal Gray Stitch,Rear-wheel Drive,15–25,Gasoline,Automatic,Engine: 5.0L Ti-VCT V8
...,...,...,...,...,...,...,...,...,...,...,...
1188,15,2021 Ford F-150 Lariat,58117,58117,Lead Foot,Black,Four-wheel Drive,19–24,Gasoline,10-Speed Automatic,2.7L V6 24V PDI DOHC Twin Turbo
1189,16,2020 RAM 1500 Limited,57900,57900,Diamond Black,Black,Four-wheel Drive,19–24,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV
1190,17,2021 GMC Yukon Denali,84991,84991,Dark Sky Metallic,Jet Black,Four-wheel Drive,14–19,Gasoline,10-Speed Automatic,6.2L V8 16V GDI OHV
1191,18,2020 Subaru Ascent Touring 7-Passenger,42910,42910,Cinnamon Brown Pearl,Java Brown,All-wheel Drive,20–26,Gasoline,Automatic CVT,2.4L H4 16V GDI DOHC Turbo


In [21]:
df.to_csv('car_info.csv', encoding='utf-8')